# Feature Selection Exercise (Solution)

Adapted from Dipanjan Sarkar et al. 2018. [Practical Machine Learning with Python](https://link.springer.com/book/10.1007/978-1-4842-3207-1).

## Overview

This module covers feature selection strategies in machine learning, focusing on the main approaches used to identify and select the most relevant features for model building.

## Learning Objectives

- Understand the three main categories of feature selection methods
  - Filter methods
  - Wrapper methods
  - Embedded methods
- Learn how different feature selection strategies evaluate and select features
- Recognize when to apply different feature selection approaches

### Tasks to be completed

- Review different feature selection methodologies
- Compare filter, wrapper and embedded approaches
- Understand the tradeoffs between selection methods
- Practice implementing feature selection techniques

## Prerequisites

- Python programming environment
- Basic understanding of statistical and machine learning concepts
- Familiarity with common ML libraries

## Get Started

- Please select kernel "conda_python3" from SageMaker notebook instance.

### Import necessary libraries


In [1]:
# Import necessary dependencies and settings
import warnings  # Import the warnings module to manage and control warning messages during script execution.

import numpy as np  # Import the NumPy library and alias it as 'np' for numerical operations, especially for handling arrays and matrices.
import pandas as pd  # Import the pandas library and alias it as 'pd' for data manipulation and analysis, particularly using DataFrames.
from sklearn.datasets import load_breast_cancer  # Import the load_breast_cancer function from scikit-learn to load the breast cancer dataset for demonstration or testing.
from sklearn.ensemble import RandomForestClassifier  # Import the RandomForestClassifier class from scikit-learn for using Random Forest, an ensemble learning method for classification.
from sklearn.feature_selection import RFE, SelectKBest, VarianceThreshold, chi2  # Import feature selection techniques from scikit-learn:
#   - RFE (Recursive Feature Elimination) for feature ranking and selection by recursively fitting a model and removing the weakest features.
#   - SelectKBest for feature selection based on univariate statistical tests, selecting the top k features.
#   - VarianceThreshold for feature selection by removing features with low variance.
#   - chi2 for chi-squared statistic, often used for feature selection in classification problems with non-negative features.
from sklearn.linear_model import LogisticRegression  # Import the LogisticRegression class from scikit-learn for using logistic regression, a linear model for binary classification.
from sklearn.model_selection import cross_val_score  # Import the cross_val_score function from scikit-learn for evaluating model performance using cross-validation.

### Import _E. coli_ Dataset


In [2]:
# Define the file path for the Ecoli dataset
ecoli_data = "../../Data/ecoli.csv"

# Load the dataset into a pandas DataFrame
df = pd.read_csv(ecoli_data)

Ecoli dataset is for predicting Protein Localization Sites in Ecoli.

```
Number of Instances:  336
Number of Attributes: 8 ( 7 predictive, 1 name )
Attribute Information.
  1. Sequence Name: Accession number for the SWISS-PROT database
  2. mcg: McGeoch's method for signal sequence recognition.
  3. gvh: von Heijne's method for signal sequence recognition.
  4. lip: von Heijne's Signal Peptidase II consensus sequence score (Binary attribute).
  5. chg: Presence of charge on N-terminus of predicted lipoproteins (Binary attribute).
  6. aac: score of discriminant analysis of the amino acid content of outer membrane and periplasmic proteins.
  7. alm1: score of the ALOM membrane spanning region prediction program.
  8. alm2: score of ALOM program after excluding putative cleavable signal regions from the sequence.
Missing Attribute Values: None.
Class Distribution. The class is the localization site.
  cp  (cytoplasm)                                    143
  im  (inner membrane without signal sequence)        77
  pp  (perisplasm)                                    52
  imU (inner membrane, uncleavable signal sequence)   35
  om  (outer membrane)                                20
  omL (outer membrane lipoprotein)                     5
  imL (inner membrane lipoprotein)                     2
  imS (inner membrane, cleavable signal sequence)      2
```

You can learn more about the dataset here:

- Ecoli Dataset ([ecoli.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ecoli.data))
- Ecoli Dataset Description ([ecoli.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ecoli.names))


## Introduction to Feature Selection Strategies

Feature selection methodologies can be categorized into three primary approaches, each with distinct characteristics:

**1. Filter Methods**  
These techniques evaluate features independently of any machine learning model, using statistical measures such as:
- Correlation coefficients (Pearson, Spearman)  
- Mutual information scores  
- Chi-squared tests  
- Variance thresholds  

Filter methods are computationally efficient but may overlook feature interactions.

**2. Wrapper Methods**  
Wrapper approaches iteratively assess feature subsets by:
- Building models with different feature combinations  
- Evaluating performance metrics  
- Selecting optimal subsets (e.g., via recursive feature elimination)  

While more computationally intensive, they better capture feature interdependencies.

**3. Embedded Methods**  
These hybrid techniques integrate feature selection within model training:
- Tree-based importance scores (Gini, entropy)  
- Regularization parameters (L1/L2 penalties)  
- Model-specific coefficients  

Popular implementations include Random Forest feature importance and LASSO regression, offering a balance between efficiency and interaction awareness.


In [3]:
# Configure NumPy to suppress scientific notation when printing floating point numbers.
# This ensures that numbers close to zero are displayed as 0 instead of in scientific notation.
np.set_printoptions(suppress=True)

# Retrieve the current print options and store the value of the "threshold" setting.
# The threshold determines the number of array elements printed before summarization with "...".
pt = np.get_printoptions()["threshold"]

## Threshold-Based Methods

Threshold-based feature selection is a filter approach that retains features meeting specified statistical criteria while discarding others. Common thresholds include:  
- **Variance thresholds** (removing low-variance features)  
- **Score cutoffs** (e.g., keeping features above a percentile in chi-squared/MI tests)  
- **Cardinality limits** (capping maximum features selected)  

This method provides computational efficiency but requires careful threshold tuning to balance feature reduction with information retention.

### Variance-Based Thresholding

Variance-based feature selection eliminates low-variance features that provide minimal predictive value, as their near-constant values contribute little to model differentiation. By setting a user-defined threshold (e.g., `threshold=0.1`), features with variance below this cutoff are automatically removed. This approach is particularly effective for:  
- **High-dimensional datasets** with many zero-variance features  
- **Preprocessing pipelines** where scale-invariant filtering is needed  
- **Noise reduction** by eliminating quasi-constant variables  

In [4]:
# Get the shape (number of rows and columns) of the DataFrame
df.shape  # Returns a tuple (num_rows, num_columns)

# Convert the categorical variable "site" into dummy/indicator variables,
# creating a separate binary column for each unique category in "site".
ecoli_site = pd.get_dummies(df["site"])

# Display the first few rows of the newly created dummy variables
ecoli_site.head()

,cp,im,imL,imS,imU,om,omL,pp
0,False,True,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False
3,True,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False


In [5]:
# Create a VarianceThreshold object to filter out low-variance features  
# Features with a variance lower than 0.15 will be removed  
vt = VarianceThreshold(threshold=0.15)  

# Fit the VarianceThreshold object to the dataset (ecoli_site)  
# This calculates the variance of each feature  
vt.fit(ecoli_site)  


VarianceThreshold(threshold=0.15)

In [6]:
# Create a DataFrame to display the variance of each feature and whether it was selected.  
pd.DataFrame(
    {   
        "variance": vt.variances_,  # Store the variance values of the features.
        "select_feature": vt.get_support()  # Boolean mask indicating selected features.
    },  
    index=ecoli_site.columns,  # Use feature names as index for better readability.
).T  # Transpose the DataFrame to display features as columns.

,cp,im,imL,imS,imU,om,omL,pp
variance,0.244464,0.176649,0.005917,0.005917,0.093316,0.055981,0.01466,0.130811
select_feature,True,True,False,False,False,False,False,False


In [7]:
# Get the final subset of selected features  
# `vt.get_support()` returns a boolean mask indicating which features were selected  
# `iloc[:, vt.get_support()]` filters the dataset to keep only the selected features  
# `head()` retrieves the first few rows for preview  
ecoli_site_subset = ecoli_site.iloc[:, vt.get_support()].head()  
ecoli_site_subset  

,cp,im
0,False,True
1,True,False
2,False,False
3,True,False
4,False,False


## Diagnostic Breast Cancer dataset

This dataset is known as the Wisconsin
Diagnostic Breast Cancer dataset, which is also available in its native or raw format at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic), which is the UCI Machine Learning
repository.


In [8]:
# Load the breast cancer dataset from sklearn
bc_data = load_breast_cancer()

# Convert feature data into a Pandas DataFrame with appropriate column names
bc_features = pd.DataFrame(bc_data.data, columns=bc_data.feature_names)

# Convert target labels into a DataFrame with a descriptive column name
bc_classes = pd.DataFrame(bc_data.target, columns=["IsMalignant"])

# Build the feature set (X) and response class labels (y)
bc_X = np.array(bc_features)  # Convert features DataFrame to a NumPy array
bc_y = np.array(bc_classes).T[0]  # Convert target labels to a NumPy array and flatten it

# Print the shape of the feature set and response class labels for verification
print("Feature set shape:", bc_X.shape)
print("Response class shape:", bc_y.shape)

Feature set shape: (569, 30)
Response class shape: (569,)


In [9]:
# Set the print threshold to limit the number of array elements displayed  
np.set_printoptions(threshold=30)

# Print the shape of the feature set  
print("Feature set data [shape: " + str(bc_X.shape) + "]")

# Print the feature set with values rounded to 2 decimal places  
print(np.round(bc_X, 2), "\n")

# Print the feature names  
print("Feature names:")
print(np.array(bc_features.columns), "\n")

# Print the shape of the predictor class label data  
print("Predictor Class label data [shape: " + str(bc_y.shape) + "]")

# Print the predictor class label data  
print(bc_y, "\n")

# Print the predictor class name  
print("Predictor name:", np.array(bc_classes.columns))

# Reset the print threshold to its previous value (stored in variable 'pt')  
np.set_printoptions(threshold=pt)


Feature set data [shape: (569, 30)]
[[ 17.99  10.38 122.8  ...   0.27   0.46   0.12]
 [ 20.57  17.77 132.9  ...   0.19   0.28   0.09]
 [ 19.69  21.25 130.   ...   0.24   0.36   0.09]
 ...
 [ 16.6   28.08 108.3  ...   0.14   0.22   0.08]
 [ 20.6   29.33 140.1  ...   0.26   0.41   0.12]
 [  7.76  24.54  47.92 ...   0.     0.29   0.07]] 

Feature names:
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension'] 

Predictor Class label data [shape: (569,)]
[0 0 0 ... 0 0 1] 

Predictor name: ['IsMaligna

The dataset contains a binary response variable where 1 represents benign tumors and 0 indicates malignant cases, providing clear clinical distinction for classification tasks. Accompanying this target variable are 30 continuous-valued features derived from digitized images of breast mass biopsies. These numerical features quantitatively characterize various morphological properties of cell nuclei, including size, texture, and structural characteristics, which are critical for pathological assessment. The real-valued nature of these predictors suggests they may benefit from standardization before modeling, while the binary classification task lends itself to evaluation using metrics like ROC-AUC and F1-score given the potential clinical consequences of misclassification.

In [10]:
# Use the chi-square test to evaluate the importance of features  
# and select the top 15 best features out of the 30 available features.
skb = SelectKBest(score_func=chi2, k=15)  

# Fit the SelectKBest model to the feature matrix (bc_X) and target variable (bc_y)  
# to determine the most relevant features based on the chi-square test.
skb.fit(bc_X, bc_y)

SelectKBest(k=15, score_func=<function chi2 at 0x11fdcd750>)

In [11]:
# Create a list of tuples where each tuple contains a feature name and its corresponding score
feature_scores = [
    (item, score) for item, score in zip(bc_data.feature_names, skb.scores_)
]

# Sort the list of feature scores in descending order based on the score value (x[1])
# and return the top 10 most relevant features
sorted(feature_scores, key=lambda x: -x[1])[:10]

[(np.str_('worst area'), np.float64(112598.4315640537)),
 (np.str_('mean area'), np.float64(53991.65592375091)),
 (np.str_('area error'), np.float64(8758.504705334486)),
 (np.str_('worst perimeter'), np.float64(3665.035416340581)),
 (np.str_('mean perimeter'), np.float64(2011.1028637679128)),
 (np.str_('worst radius'), np.float64(491.68915743332025)),
 (np.str_('mean radius'), np.float64(266.1049171951782)),
 (np.str_('perimeter error'), np.float64(250.57189635982178)),
 (np.str_('worst texture'), np.float64(174.4493996057095)),
 (np.str_('mean texture'), np.float64(93.89750809863374))]

In [12]:
# Create a subset of the selected features obtained from our original feature set 
# using the chi-square test (via SelectKBest)
select_features_kbest = skb.get_support()  # Get the mask of selected features (True for selected)
feature_names_kbest = bc_data.feature_names[select_features_kbest]  # Get feature names corresponding to selected features

# Create a DataFrame with the selected features using the column names from the original feature set
feature_subset_df = bc_features[feature_names_kbest]

# Convert the DataFrame of selected features into a numpy array for further processing
bc_SX = np.array(feature_subset_df)

# Print the shape of the resulting feature subset array (rows, columns)
print(bc_SX.shape)

# Print the names of the selected features
print(feature_names_kbest)

(569, 15)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean concavity' 'radius error' 'perimeter error' 'area error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst compactness' 'worst concavity' 'worst concave points']


In [13]:
# Select a feature subset of the Wisconsin Diagnostic Breast Cancer dataset using chi-square tests
# The 'feature_subset_df' contains the result of feature selection based on chi-square tests
# We are selecting a specific subset of rows (from row 20 to 25) and rounding the values to 2 decimal places
np.round(feature_subset_df.iloc[20:25], 2)

,mean radius,mean texture,mean perimeter,mean area,mean concavity,radius error,perimeter error,area error,worst radius,worst texture,worst perimeter,worst area,worst compactness,worst concavity,worst concave points
20,13.08,15.71,85.63,520.0,0.05,0.19,1.38,14.67,14.50,20.49,96.09,630.5,0.28,0.19,0.07
21,9.50,12.44,60.34,273.9,0.03,0.28,1.91,15.70,10.23,15.66,65.13,314.9,0.11,0.09,0.06
22,15.34,14.26,102.50,704.4,0.21,0.44,3.38,44.91,18.07,19.08,125.10,980.9,0.60,0.63,0.24
23,21.16,23.04,137.20,1404.0,0.11,0.69,4.30,93.99,29.17,35.59,188.00,2615.0,0.26,0.32,0.20
24,16.65,21.38,110.00,904.6,0.15,0.81,5.46,102.60,26.46,31.56,177.00,2215.0,0.36,0.47,0.21


To evaluate the impact of feature selection, we will train and compare two logistic regression models: one using the full set of 30 features and another with only the 15 selected features. Model performance will be assessed via accuracy (percentage of correct predictions) under a five-fold cross-validation scheme, ensuring robust evaluation while mitigating overfitting. This comparative analysis will isolate the effect of feature reduction, revealing whether the selected subset preserves predictive power or potentially improves model generalizability through noise reduction. The experiment’s design specifically controls for algorithmic bias by maintaining identical evaluation metrics and validation protocols across both feature sets.

In [14]:
# Suppress warnings to keep the output clean (e.g., avoid displaying convergence warnings)
warnings.filterwarnings("ignore")

# Build a Logistic Regression model with a maximum of 1000 iterations for convergence
lr = LogisticRegression(max_iter=1000)

# Evaluate accuracy of the model built on the complete feature set using 5-fold cross-validation
# 'cross_val_score' performs cross-validation and calculates accuracy for each fold
full_feat_acc = np.average(cross_val_score(lr, bc_X, bc_y, scoring="accuracy", cv=5))

# Evaluate accuracy of the model built on the selected feature set using 5-fold cross-validation
sel_feat_acc = np.average(cross_val_score(lr, bc_SX, bc_y, scoring="accuracy", cv=5))

# Print out the results
print("Model accuracy statistics with 5-fold cross validation")
# Output the accuracy of the model using the full feature set, along with the shape of the input features
print("Model accuracy with complete feature set", bc_X.shape, ":", full_feat_acc)
# Output the accuracy of the model using the selected feature set, along with the shape of the input features
print("Model accuracy with selected feature set", bc_SX.shape, ":", sel_feat_acc)

Model accuracy statistics with 5-fold cross validation
Model accuracy with complete feature set (569, 30) : 0.9525694767893185
Model accuracy with selected feature set (569, 15) : 0.9525694767893185


## Recursive Feature Elimination

Recursive Feature Elimination (RFE) is a powerful wrapper-based feature selection technique that systematically reduces features through an iterative backward elimination process. The method begins by training a specified machine learning model (such as logistic regression for classification tasks) on the complete set of features (e.g., 30 features in our case) and their corresponding target variables. RFE then evaluates and ranks features based on the model-derived importance weights (e.g., regression coefficients for linear models). In each iteration, the lowest-ranked features are pruned, and the model is retrained on the remaining subset. This recursive process continues until reaching a predetermined optimal number of features specified by the user. By progressively eliminating weaker features while retraining the model at each step, RFE effectively captures feature interactions and dependencies, often yielding more robust feature subsets than filter methods. This backward elimination approach is particularly valuable when working with high-dimensional datasets where identifying the most predictive feature combinations is crucial for model performance.

In [15]:
# Initialize the Logistic Regression model to be used for feature selection
lr = LogisticRegression()

# Initialize RFE (Recursive Feature Elimination) with:
# - the Logistic Regression model (lr) as the estimator
# - n_features_to_select=15, meaning we want to select the top 15 features
# - step=1, which means we remove one feature at a time in each iteration
rfe = RFE(estimator=lr, n_features_to_select=15, step=1)

# Fit the RFE model on the breast cancer dataset (X: features, y: target)
rfe.fit(bc_X, bc_y)

RFE(estimator=LogisticRegression(), n_features_to_select=15)

In [16]:
# Obtain the final selected features from RFE (Recursive Feature Elimination)
select_features_rfe = rfe.get_support()  # Get a boolean array of selected features (True for selected, False for non-selected)

# Extract the names of the features that were selected by RFE
feature_names_rfe = bc_data.feature_names[select_features_rfe]  # Filter the feature names based on the selected ones

# Print the names of the selected features
print(feature_names_rfe)

# Fit the RFE model to the data (this step might be part of the RFE process, not needed if already fitted)
rfe.fit(bc_X, bc_y)  # Fit the RFE model to the input data (bc_X) and target labels (bc_y)

['mean radius' 'mean compactness' 'mean concavity' 'mean concave points'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'worst texture' 'worst perimeter' 'worst smoothness' 'worst compactness'
 'worst concavity' 'worst concave points' 'worst symmetry']


RFE(estimator=LogisticRegression(), n_features_to_select=15)

In [17]:
# Compare the feature subset selected using SelectKBest (feature_names_kbest) 
# with the feature subset selected using Recursive Feature Elimination (RFE) 
# (feature_names_rfe). The intersection will show the common features between 
# these two subsets.
set(feature_names_kbest) & set(feature_names_rfe)

{np.str_('area error'),
 np.str_('mean concavity'),
 np.str_('mean radius'),
 np.str_('perimeter error'),
 np.str_('radius error'),
 np.str_('worst compactness'),
 np.str_('worst concave points'),
 np.str_('worst concavity'),
 np.str_('worst perimeter'),
 np.str_('worst texture')}

## Model based selection

Tree-based machine learning models, including individual decision trees and ensemble methods like random forests, serve dual purposes in predictive analytics: both as powerful modeling techniques and as effective feature selection tools. These models inherently calculate feature importance scores during their training process—through metrics like Gini impurity reduction or mean decrease in accuracy—which quantitatively measure each feature's contribution to prediction accuracy. By analyzing these importance scores, data scientists can objectively identify and retain the most predictive features while eliminating irrelevant or redundant ones. This embedded selection approach offers key advantages over standalone feature selection methods: it efficiently captures complex feature interactions, scales well to high-dimensional data, and produces importance rankings that are directly optimized for the specific modeling task. The selected features often yield more interpretable models while maintaining or even improving predictive performance compared to using the full feature set.

In [18]:
# Importing the RandomForestClassifier model from sklearn
rfc = RandomForestClassifier()  # Initialize a RandomForestClassifier object

# Fitting the model on the training data (bc_X for features, bc_y for labels)
rfc.fit(bc_X, bc_y)  # Train the random forest model on the given feature set (bc_X) and target labels (bc_y)

RandomForestClassifier()

In [19]:
# Use random forest estimator to score the features based on their importance
# The `rfc.feature_importances_` attribute provides the importance scores for each feature
importance_scores = rfc.feature_importances_

# Create a list of tuples where each tuple contains a feature name and its corresponding importance score
feature_importances = [
    (feature, score) for feature, score in zip(bc_data.feature_names, importance_scores)
]

# Sort the list of feature importances by the score in descending order and select the top 10 features
# The `lambda x: -x[1]` ensures the list is sorted in descending order based on the score
sorted(feature_importances, key=lambda x: -x[1])[:10]

[(np.str_('worst radius'), np.float64(0.1515240224856463)),
 (np.str_('worst perimeter'), np.float64(0.12948847996672794)),
 (np.str_('worst concave points'), np.float64(0.11815142406548058)),
 (np.str_('worst area'), np.float64(0.10935604611182731)),
 (np.str_('mean concave points'), np.float64(0.07456044954323722)),
 (np.str_('mean perimeter'), np.float64(0.06664957902529337)),
 (np.str_('mean area'), np.float64(0.048822415540674166)),
 (np.str_('mean concavity'), np.float64(0.04220631569496321)),
 (np.str_('mean radius'), np.float64(0.04067253014332115)),
 (np.str_('area error'), np.float64(0.03435460579612631))]

You can now use a threshold based parameter to filter out the top n features as needed or you can even
make use of the SelectFromModel meta-transformer provided by scikit-learn by using it as a wrapper on
top of this model.


## Conclusion

Feature selection is a critical step in machine learning that helps identify the most relevant features for model building. We learned about three main approaches:

- Filter methods that use metrics and statistical tests
- Wrapper methods that evaluate feature subsets recursively
- Embedded methods that combine benefits of other approaches using ML models

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.
